In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_ollama import ChatOllama
from typing import TypedDict

In [17]:
class AgentState(TypedDict):
    name: str
    age: str
    skils: list[str]
    result: str


In [29]:
def greeting_node(state:AgentState)->AgentState:
    greeting = f"{state['name']}, welcome to the system!"
    return {"result":greeting}

In [38]:
def age_node(state:AgentState)->AgentState:
    user_age=f"You are {state['age']} years old!"
    return {"result": state["result"] + user_age}

In [41]:
# def skills_node(state:AgentState)->AgentState:
#     user_skills= "You have skills in: "+ " ".join(state["skils"])
#     return {"result": state['result']+ user_skills}

def skills_node(state: AgentState) -> AgentState:
    skills = state["skils"]          # ← make sure the key is spelled correctly!
    
    if not skills:                              # no skills listed
        skills_str = "no listed skills"
    elif len(skills) == 1:                      # exactly one skill
        skills_str = skills[0]
    elif len(skills) == 2:                      # two skills → “A and B”
        skills_str = f"{skills[0]} and {skills[1]}"
    else:                                       # three or more → “A, B, …, and Z”
        skills_str = ", ".join(skills[:-1]) + f" and {skills[-1]}"
    
    user_skills = f"You have skills in: {skills_str}"
    return {"result": state["result"] + " " + user_skills}


In [42]:
graph= StateGraph(AgentState)

graph.add_node("greeting",greeting_node)
graph.add_node("user_age",age_node)
graph.add_node("skills",skills_node)

graph.add_edge(START, "greeting")
graph.add_edge("greeting","user_age" )
graph.add_edge("user_age","skills")
graph.add_edge("skills", END)

workflow=graph.compile()

In [43]:
initial_state={"name":"Linda", "age": "31", "skils": ["Python", "Machine Learning", "LangGraph"]}

workflow.invoke(initial_state)

{'name': 'Linda',
 'age': '31',
 'skils': ['Python', 'Machine Learning', 'LangGraph'],
 'result': 'Linda, welcome to the system!You are 31 years old! You have skills in: Python, Machine Learning and LangGraph'}

In [186]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document
from typing_extensions import List, TypedDict
from pydantic import BaseModel, Field
from langchain_core.messages import BaseMessage
from langchain_core.messages import SystemMessage
from langchain import hub
import textwrap
import os
class AgentState2(TypedDict):
    question: str
    answer: str
    context:List[Document]


In [170]:
class EvaluationSchema(BaseModel):
    answer:str= Field(description="answer of user asked question")

In [171]:
llm = ChatOllama(model="llama3.2")
structured_llm=llm.with_structured_output(EvaluationSchema)

In [173]:
def retriever(state:AgentState2)->AgentState2:
    base_dir = os.getcwd()
    product_file_path = os.path.join(base_dir, "./product_help.txt")
    product_loader = TextLoader(product_file_path)
    product_docs = product_loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=300,
            chunk_overlap=50
        )
    product_chunks = text_splitter.split_documents(product_docs)
    for doc in product_chunks:
            doc.metadata["type"] = "product"
    docs=product_chunks
    embeddings = OllamaEmbeddings(
            model="mxbai-embed-large",
            base_url="http://10.28.85.206:11434/"
        )
    vectorstore = Chroma.from_documents(
            documents=docs,
            embedding=embeddings,
            persist_directory="chroma_db",
        )
    vectorstore.persist()
    retrieved_docs = vectorstore.similarity_search(state["question"], k=5)
    
    return {"context":retrieved_docs}

In [193]:
def generate(state:AgentState2)->AgentState2:
    rag_prompt = hub.pull("rlm/rag-prompt")
    extra_rules = textwrap.dedent("""\
        Answer requirements:
        1. If any link is provided, wrap it in an anchor tag with target="_blank".
        2. Only use links that appear in the knowledge text files.
        3. Do not invent links.

        If the question is not related to accounting or the GL website, reply:
        "Please ask a question related to accounting or the GL website."

        Concise accounting answer:
    """)
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    base_prompt = rag_prompt.invoke({"question": state["question"], "context": docs_content})
    full_prompt = f"{base_prompt}\n\n{extra_rules}"
    answer=structured_llm.invoke(full_prompt)
    return {'answer': answer}
    

In [191]:
graph= StateGraph(AgentState2)

graph.add_node("retriever", retriever)
graph.add_node("generate",generate)

graph.add_edge(START, "retriever")
graph.add_edge("retriever","generate")
graph.add_edge("generate", END)

workflow2 = graph.compile()


In [192]:
final_state=workflow2.invoke({"question":"How do I create a new account (sign up)?"})

/media/muhammadahmad/DATA5/data-derive/generative-ai/chatboot/.venv/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


answer answer='To sign up for a new account, go to <a href="https://gl.vteamslabs.com/signup" target="_blank">https://gl.vteamslabs.com/signup</a>.\nFill out the registration form with your name, email, and desired password.'


In [185]:
print(final_state["answer"])

answer='To sign up for a new account, go to <a href="https://gl.vteamslabs.com/signup" target="_blank">https://gl.vteamslabs.com/signup</a>.\n\nFill out the registration form with your name, email, and desired password.'
